### Creating representative observations for Clyfar and the Uinta Basin

In [ ]:
import os

from obs.obsdata import ObsData

In [1]:
df, meta_df = ObsData.load_dfs("../data", "UB_obs_2020.parquet")
df.head(10)

NameError: name 'ObsData' is not defined